In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab
import torch
import sys
sys.path.insert(1, '/home/nicolas/code/src')
sys.path.insert(1, '/home/nicolas/code/data')


from scipy.integrate import simps
from scipy import stats
import h5py

import gzip
import pickle

from TMCRBM2D import TMCRBM2D
device = torch.device("cuda")
dtype = torch.float
torch.set_num_threads(4)

In [2]:
f = gzip.open('../data/mnist.pkl.gz', 'rb')
u = pickle._Unpickler(f)
u.encoding = 'latin1'
p = u.load()
train_set, _, _ = p

X = torch.as_tensor(train_set[0][:10000,:].T, device = device, dtype = dtype)
X.shape

torch.Size([784, 10000])

In [3]:
fname = '../model/AllParametersTESTCV2_LongRUNExMC_NS10000_TEMP_0_MNIST_Nh500_lr0.01_l20_NGibbs10000.h5'

In [4]:
Nv = 784
Nh = 500
verbose = 0
save_fig = False

lr = 0.01
NGibbs = 70
it_mean = 20

mb_s = 1000
num_pcd = 1000
ep_max = 50
N = 20000
nb_chain = 10
nb_point_dim = torch.tensor([50,50])

stamp = 'MNIST_TMC2DRBM_NGibbs_'+str(NGibbs)+'_Nh'+str(Nh)+'_Nv' + str(Nv)+'_Nmb'+str(mb_s)+'_Nepoch'+str(ep_max)+'_lr_'+str(lr) + '_N' + str(N) + '_Npoint' + str(nb_point_dim.prod().item()) + '_Nchain' + str(nb_chain)
myRBM = TMCRBM2D(num_visible=Nv,
            num_hidden=Nh,
            device=device,
            lr=lr,
            gibbs_steps=NGibbs,
            UpdCentered=True,
            mb_s=mb_s,
            num_pcd=num_pcd,
            ResetPermChainBatch=False,
            CDLearning=True,
            N = N,
            nb_chain = nb_chain,
            nb_point_dim=nb_point_dim,
            verbose=verbose,
            save_fig = save_fig
            )


In [5]:
fq_msr_RBM = 10
myRBM.file_stamp = stamp
base = 1.7
v = np.array([0,1],dtype=int)
allm = np.append(np.array(0),base**np.array(list(range(30))))
for k in range(30):
    for m in allm:
        v = np.append(v,int(base**k)+int(m)) 
v = np.array(list(set(v)))
v = np.sort(v)
myRBM.list_save_time = v

v = np.array(list(set(v)))
v = np.sort(v)
myRBM.list_save_time = v
myRBM.list_save_rbm = np.arange(1, ep_max, fq_msr_RBM)

myRBM.list_save_rbm = np.arange(1,ep_max,fq_msr_RBM)
myRBM.fit(X, ep_max)

IT  0
Initialisation time :  0.32041215896606445


KeyboardInterrupt: 